In Context evaluation of the best performing models.

We now wish to evaluate each of our projection models at the token level. We construct an evaluation dataset from a set of homonyms from the McRae feature set. The dataset consists of tokens of homonymous words collected from semcor, alongside gold feature data (mcrae) for disambiguated senses.

We have gold features in mcrae and buchanan format, so we only use these datasets, not binder.


We have trained models with best performing hyperparameters on the other words in the datasets.

4 layer ffnn (1k)
4 layer ffnn (5k)
4 layer ffnn (glove)
plsr (1k)
plsr (5k)
plsr(glove)
label propagation (1k)
label propagation (5k)
label propagation (glove)

To evaluate a model, we use it to predict features in context for each token in the dataset. Then, we compare to the gold feature data. We also calculate the average cosine distance between gold and predicted vectors. 

In [1]:
import sys
sys.path.append("../src/")
sys.path.append("../")

import torch
from bert import *
from feature_data import *
from multiprototype import *
from models import *
from utils import *
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Lemma
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt


from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

from scipy.stats import spearmanr

bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmp_8_eamj0
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

We want to evaluate the model on its ability to predict features in context. First, we obtain gold vectors for both senses of 10 ambiguous words from the McRae et al. data

What are the words?

# Construct the Evaluation Dataset.

In [6]:
save_path = '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters'
model = torch.load(save_path)

df = pd.read_csv('../data/processed/bnc_contexts_for_mcrae_homonyms.csv')
feature_norms = BuchananFeatureNorms('../data/external/buchanan/cue_feature_words.csv')



evaluate_in_context(model, df, bert, feature_norms)

NameError: name 'evaluate_in_context' is not defined

In [7]:
ambiguous_pairs = [
    ('bat_animal', 'bat_baseball'),
    ('board_wood', 'board_black'),
    ('bow_ribbon', 'bow_weapon'),
    ('cap_bottle', 'cap_hat'),
    #('crane_machine', 'crane_animal')
    ('hose', 'hose_leggings'),
    ('mink', 'mink_coat'), # this one is not fully disambiguated
    ('mouse', 'mouse_computer'),
    ('pipe_smoking', 'pipe_plumbing'),
    ('tank_army', 'tank_container')
]

In [8]:
[item for t in ambiguous_pairs for item in t]


['bat_animal',
 'bat_baseball',
 'board_wood',
 'board_black',
 'bow_ribbon',
 'bow_weapon',
 'cap_bottle',
 'cap_hat',
 'hose',
 'hose_leggings',
 'mink',
 'mink_coat',
 'mouse',
 'mouse_computer',
 'pipe_smoking',
 'pipe_plumbing',
 'tank_army',
 'tank_container']

In [10]:
"""
take a look at the gold features for these words.
"""
norms = BuchananFeatureNorms('../data/external/buchanan/cue_feature_words.csv')


norms.print_features('bat')
norms.print_features('bat_baseball')
norms.print_features('bat_animal')

features for bat: {'animal': 6.779661017, 'cricket': 7.344632768, 'fly': 7.90960452, 'hit': 7.344632768, 'wood': 9.039548023}
features for bat_baseball: {'ball': 60.0, 'heave': 16.66666667, 'hit': 96.66666667, 'long': 50.0, 'metal': 53.33333333, 'sport': 23.33333333, 'swing': 20.0, 'wood': 73.33333333}
features for bat_animal: {'animal': 33.33333333, 'black': 53.33333333, 'blind': 33.33333333, 'cave': 46.66666667, 'down': 33.33333333, 'fang': 23.33333333, 'fly': 63.33333333, 'fur': 36.66666667, 'mammal': 23.33333333, 'navigate': 30.0, 'nocturnal': 63.33333333, 'radar': 30.0, 'scare': 20.0, 'screech': 26.66666667, 'sleep': 33.33333333, 'small': 20.0, 'vampire': 16.66666667, 'wing': 86.66666667}


Now we want to find examples of each sense and make predictions using the model for that word. We will score the predictions of each example and then average them. Here are 10 examples of bat_animal. Load these from the file we have created with these words.

# Train the Models

Hopefully this is done already 

In [50]:
"""
models and save paths

"""
buchanan_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters',
    '../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.buchanan.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.5k',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.1k',
]

buchanan_glove_models = [
    '../trained_models/model.plsr.buchanan.allbuthomonyms.glove.300components.300max_iters',
    #'../trained_models/model.ffnn.buchanan.allbuthomonyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.modabs.buchanan.allbuthomoyms.glove'   
]

mcrae_models = [
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.mc_rae_real.allbuthomonyms.5k.100components.500max_iters',
    '../trained_models/model.plsr.mc_rae_real.allbuthomonyms.1k.50components.500max_iters',
    '../trained_models/model.modabs.mc_rae_real.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
]

mcrae_glove_models = [
    '../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
    #'../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
]

binder_models = [
    '../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.binder.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.5k.300components.500max_iters',
    '../trained_models/model.plsr.binder.1k.300components.500max_iters',
    '../trained_models/model.modabs.binder.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.binder.1k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
]

binder_glove_models = [
    '../trained_models/model.ffnn.binder.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.binder.glove.300components.500max_iters',
    '../trained_models/model.modabs.binder.glove.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',    
]

# Run the models on the data

TODO are you sure this is the exact dataset you want? Per Katrins comment:

Another small question on the McRae homonym analysis: If there was a word labeled with sense cap.n.02 but it's not "cap" but another word, say "lid", do you also use that in this analysis, or only the correct lemma?

In [38]:
pd.set_option('display.max_colwidth', None)
eval_words = pd.read_csv('../data/processed/mcrae_homonyms_from_coca_10_20_2022.csv')
eval_words

,id,word,source,sentence,buchanan_cue_word,mcrae_cue_word
0,1,bat,COCA:2010:TV\nGood Luck Charlie,"! - Oh, real mature! - I am more mature than you are! Whatever! Oh! Okay! Okay! All right, drop the bat Or baby booboo becomes baby barbecue. You would n't dare! Try me. - Give me the baby. - Give me the bat. - The baby. - The bat. P.J.! Pull me up! Pull me up! Oh, wait! Are you quitting? - What? - You know, like, If I pull you back in, will you keep doing the sit-in? Since my alternative is",bat_baseball,bat_(baseball)
1,2,bat,COCA:2014:FIC\nBk:VengeanceIsMine,"Best's casino in the wide-open gambling town, cutting in on the Family's profits. # Enrique Sandoval's house was silent. Anthony reached the back door and found it was locked. He expected that, but it never hurt to try the simple way first. He carefully placed the bat on the Mexican tile, and unzipped his coveralls enough to slip the.45 intothe shoulder holster. That done, he dug out the set of picks he'd learned to use as a kid. The cheap General lock surrendered a moment later. The door opened into a dark, spacious",bat_baseball,bat_(baseball)
2,3,bat,COCA:2002:MAG\nPeople,"Alex looking on. In a taped confession Derek maintained that King had pushed Alex around that night, causing the boy to cry. They said they were tired of being abused and wanted to live with Chavis. They also provided a sickeningly detailed account of the assault. The sound of the bat on his father's head, said Alex, was like '' wood cracking,or hitting concrete or something. "" Earlier this year, however, the boys suddenly recanted their confession. They maintained that Chavis had induced them to say they killed King, assuring them they could claim selfdefense.",bat_baseball,bat_(baseball)
3,4,bat,COCA:2015:NEWS\nUSAToday,"happened. "" And sometimes it's better just to be quiet. Three years ago, in Kerry Wood's final season with the Cubs, he and Samardzija bet each other who could hit a baseball 100 feet away into a trash can. '' So I missed, tried to snap the bat over my leg, '' Samardzija said, '' and it didn't break.The middle of my thigh was all black and blue. I had a contusion. But I was smart enough not to tell anybody about it. It was only the second day of spring training, so I",bat_baseball,bat_(baseball)
4,5,bat,COCA:2019:NEWS\nDallas Morning News,"back half that he could n't catch up to or hit the ball on the barrel with any consistency. Odor did change his bat style to a "" cupped "" model, which means the end of the bat has been hollowed out into a cupped form. # The thinking behind the cupped bat is that it creates more balance in the wood and allows manufacturers to use aharder/denser piece of lumber. Also, there is a slight weight drop. Maybe a quarter of an ounce. Maybe a half ounce. Pretty negligible. Or maybe not. Odor did not credit the change in",bat_baseball,bat_(baseball)
...,...,...,...,...,...,...
949,30,mink,COCA:2009:FIC\nAntiochRev,"blows up. Enid wraps her fur stole tightly around her shoulders. "" Nice, "" Bonita says, stroking it. "" That real fur? "" It looked just like the ratty, old piece her friend Elnora used to wear to church which everyone knew was monkey trying to look like mink. '' I told you. I hate artificial. '' Which, Bonita notes, does n't quite answer her question. Edmund is still struggling with the lid. "" Hurry up, Ed, "" Enid says. "" Before this wind gets any stronger. "" "" What we need is",mink_coat,mink_(coat)
950,31,mink,COCA:1996:NEWS\nDenver,"$ 5.5 million to make this project work. # The goal: Weakening Colorado's environ-mental standards. # To do this, the Army has been raising minks in Minnesota and feeding them a chemical polluting the Rocky Mountain Arsenal in Colorado. Since 1991, Army officials have spent $ 8,100 per mink - or 85 times more than any one pelt is worth - to conduct thispollution experiment. # If the Minnesota minks eat enough **28;576;TOOLONG, or DIMP, without developing health problems, the Army plans to ask Colorado re

In [39]:
 eval_words = eval_words.groupby('buchanan_cue_word').apply(lambda x: x.sample(10)).reset_index(drop=True)

In [40]:


"""
data is a list of tuples of (cue word, lemma, context)

for each model, we want to run an analysis of these
"""

    
def evaluate_in_context(model, df, bert, glove=False, model_name=None, dataset='mc_rae'):
    """
    df has columns
    sentence
    buchanan_cue_word
    mcrae_cue_word
    """
    n = 0
    
    cosines = []
    top_10_precs = []
    top_20_precs = []
    top_k_precs = []
    correlations = []
    rows = []

    num_top_10 = 0
    num_top_20 = 0
    num_total = 0

    for index, row in df.iterrows():
        if dataset == 'buchanan':
            cue_word = row.buchanan_cue_word  
        else:
            cue_word = row.mcrae_cue_word  

        context = row.sentence
        word = row.word
        
        n +=1


        prediction = model.predict_in_context(word, context, bert, glove=glove)
        #top_k =  model.predict_top_n_features_in_context(singular, context, k, bert)

        # get gold feature
        gold_vector = model.feature_norms.get_feature_vector(cue_word)
        gold_feats = model.feature_norms.get_features(cue_word)
        k = len(gold_feats)

        # get cosines
        cos = 1 - cosine(prediction, gold_vector)
        cosines.append(cos)

        # get MAP at 10
        top_10 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
        top_10_gold = model.feature_norms.top_n(cue_word, 10)
        
        num_in_top_10 = len(set(top_10).intersection(set(top_10_gold)))
        top_10_prec = num_in_top_10 / len(top_10_gold)
        top_10_precs.append(top_10_prec)

        # get MAP at 20
        top_20 = model.predict_top_n_features_in_context(word, context, 10, vec=prediction, bert=bert)
        top_20_gold = model.feature_norms.top_n(cue_word, 20)

        num_in_top_20 = len(set(top_20).intersection(set(top_20_gold)))
        top_20_prec = num_in_top_20 / len(top_20_gold)
        top_20_precs.append(top_20_prec)

        # get MAP at k
        gold_len = len(gold_feats)
        top_k = model.predict_top_n_features_in_context(word, context, gold_len, vec=prediction, bert=bert)
        num_in_top_k = len(set(top_k).intersection(set(gold_feats)))
        top_k_prec = num_in_top_k / gold_len
        top_k_precs.append(top_k_prec)

        # get correlation
        corr, p = spearmanr(prediction, gold_vector)
        correlations.append(corr)

        #if ((i % 20 ==0) and debug=='info') or (debug=='true'):
#         if False:
#             print(word)
#             print("top ten predicted features: ", top_10)
#             print("top ten gold features: ", top_10_gold)
#             print("gold features: ", gold_feats)
#             print("top k predicted features: ", top_k)

#             print("cosine: %f" % cos)
#             print("correlation: %f" % corr)
#             print("top k prec: %f" % top_k_prec)
    
    
        row = (cue_word, context, word, top_10_prec, top_20_prec, top_k_prec, top_10, top_10_gold, top_k, gold_feats, cos)
        rows.append(row)
    
    cols = ['cue_word', 
               'context', 
               'lemma', 
               'top_10_prec', 
               'top_20_prec', 
               'top_k_prec',
               'top_10', 
               'top_10_gold', 
               'top_k', 
               'gold_feats', 
               'cos']
    
        
    avg_top_10_prec = np.average(top_10_precs)
    avg_top_20_prec = np.average(top_20_precs)
    avg_top_k_prec = np.average(top_k_precs)
    average_correlation = np.average(correlations)
    average_cosine = np.average(cosines)
        
        
    print("Average cosine between gold and predicted feature norms: %s" % average_cosine)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector: ", top_10_prec)
    print("average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector: ", top_20_prec)
    print("Average % @k (derby metric)", top_k_prec)
    print("correlation between gold and predicted vectors: %s " % average_correlation)        
    print("total number of predictions: ", n)
    
    results_df = pd.DataFrame.from_records(rows, columns = cols)
    results_df['avg_top_10_prec'] = avg_top_10_prec
    results_df['avg_top_20_prec'] = avg_top_20_prec
    results_df['avg_top_k_prec'] = avg_top_k_prec
    results_df['avg_correlation'] = average_correlation
    results_df['avg_cosine'] = average_cosine
    
    # TODO make this possible!!!!
    # results_df['model'] = model.name ()
    results_df['model'] = model_name

    
    return results_df
    



# Evaluate McRae models

In [41]:
dfs = []

for save_path in mcrae_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)

    df = evaluate_in_context(model, eval_words, bert, model_name=save_path, dataset='mc_rae')
    dfs.append(df)
    
# need to do this separately because we dont have nice self examining attributes for our model classes to tell when its 
# been trained on glove
for save_path in mcrae_glove_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path, dataset='mc_rae')
    dfs.append(df)

df = pd.concat(dfs)
df.to_csv('../results/homonymous_words_eval_mcrae_models_expanded_10_20_22.csv')

****************************************
*** Evaluating ../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
Average cosine between gold and predicted feature norms: 0.24112600584701804
average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector:  0.0
average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector:  0.0
Average % @k (derby metric) 0.0
correlation between gold and predicted vectors: 0.055783235473536016 
total number of predictions:  130
****************************************
*** Evaluating ../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.1k.50epochs.0.5dropout.lr1e-4.hsize300 model ***
****************************************
Average cosine between gold and predicted feature norms: 0.25036146954868194
average Percentage (%) of gold gold-standard features retrieved in the t

In [48]:
df.head(100)

,cue_word,context,lemma,top_10_prec,top_20_prec,top_k_prec,top_10,top_10_gold,top_k,gold_feats,cos,avg_top_10_prec,avg_top_20_prec,avg_top_k_prec,avg_correlation,avg_cosine,model
0,bat_(animal),"causes and consequences of sociality in animals. Detailed studies on bat sociality are rare, however, when compared with the information available for other social mammals, such as primates, carnivores, ungulates, and rodents. Modern field technologies and new molecular methods are now providing opportunities to study aspects of bat biology that were previously inaccessible. Consequently, bat social systems are emerging as farmore complex than had been imagined. Variable dispersal patterns, complex olfactory and acoustic communication, flexible context-related interactions, striking cooperative behaviors, and cryptic colony structures in the form of fission-fusion systems have been documented. Bat",bat,0.3,0.3125,0.3125,"[is_furry, an_animal, a_mammal, is_large, has_4_legs, has_legs, is_black, is_small, an_insect, has_fur]","[an_animal, beh_-_sleeps_upside_down, is_blind, has_fur, lives_in_caves, is_black, beh_-_flies, beh_-_is_nocturnal, has_wings, associated_with_vampires]","[beh_-_eats, a_carnivore, a_musical_instrument, is_furry, is_large, has_legs, has_4_legs, an_animal, a_mammal, has_fur, has_a_tail, is_brown, is_black, is_small, an_insect, is_long]","[has_wings, beh_-_flies, beh_-_is_nocturnal, is_black, lives_in_caves, has_fur, an_animal, beh_-_sleeps_upside_down, is_blind, beh_-_uses_radar_to_navigate, beh_-_screeches, a_mammal, has_fangs, is_scary, is_small, associated_with_vampires]",0.323878,0.215021,0.201111,0.213409,0.055783,0.241126,../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
1,bat_(animal),"and as a consequence the resultant bat swarms are transient, have a flexible composition, and comprise mostly unrelated bats (Kerth et al. 2003a). How bats from separated colonies find swarming sites that are often located dozens of kilometers away from the maternity colonies remains one of the puzzles of bat biology. # Several cooperative behaviors in bats are linked to communal breeding. Mutualwarming and babysitting of pups have thus far been documented in only a few species, such as Antrozous pallidus (figure 3a; Wilkinson 1988, Lewis 1996, Zubaid et al. 2006, Willis and Brigham 2007",bat,0.4,0.3125,0.3750,"[is_green, has_wings, is_large, has_legs, is_long, an_animal, beh_-_flies, an_insect, is_small, is_black]","[an_animal, beh_-_sleeps_upside_down, is_blind, has_fur, lives_in_caves, is_black, beh_-_flies, beh_-_is_nocturnal, has_wings, associated_with_vampires]","[has_4_legs, has_fur, a_musical_instrument, beh_-_stings, has_wings, is_large, is_green, beh_-_flies, an_animal, is_long, has_legs, made_of_wood, beh_-_crawls, an_insect, is_small, is_black]","[has_wings, beh_-_flies, beh_-_is_nocturnal, is_black, lives_in_caves, has_fur, an_animal, beh_-_sleeps_upside_down, is_blind, beh_-_uses_radar_to_navigate, beh_-_screeches, a_mammal, has_fangs, is_scary, is_small, associated_with_vampires]",0.333446,0.215021,0.201111,0.213409,0.055783,0.241126,../trained_models/model.ffnn.mc_rae_real.allbuthomonyms.5k.50epochs.0.5dropout.lr1e-4.hsize300
2,bat_(animal),"blades. Why would bats be around wind turbines in the first place? And which species were most at risk? "" It took all of us by surprise "" says Cryan, a research biologist at the US Geological Survey's Fort Collins Science Center in Colorado. Cryan was speaking of the bat community's reaction to news that hundreds of the flying mammals had died one nightat Mountaineer Wind Energy Center in Thomas, West Virginia, and, later, at other wind farms throughout the United States, Canada, and Europe. "" The researchers at Mountaineer were looking for birds, not",bat,0.4,0.3125,0.3750,"[has_legs, a_musical_instrument, beh_-_flies, has_wings, is_green, an_animal, is_long, is_small, is_black, an_insect]","[an_animal, beh_-_sleeps_upside_dow

# Evaluate Buchanan models

In [49]:
dfs = []

for save_path in buchanan_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, model_name=save_path, dataset='buchanan')
    dfs.append(df)

for save_path in buchanan_glove_models:
    print("****************************************")
    print("*** Evaluating %s model ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    df = evaluate_in_context(model, eval_words, bert, glove=True, model_name=save_path, dataset='buchanan')
    dfs.append(df)


df = pd.concat(dfs)
df.to_csv('../results/homonymous_words_eval_buchanan_models_expanded_10_20_22.csv')

****************************************
*** Evaluating ../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters model ***
****************************************
Average cosine between gold and predicted feature norms: 0.31045676424942326
average Percentage (%) of gold gold-standard features retrieved in the top 10 features of the predicted vector:  0.3
average Percentage (%) of gold gold-standard features retrieved in the top 20 features of the predicted vector:  0.23076923076923078
Average % @k (derby metric) 0.3076923076923077
correlation between gold and predicted vectors: 0.07929599521142182 
total number of predictions:  130
****************************************
*** Evaluating ../trained_models/model.plsr.buchanan.allbuthomonyms.1k.300components.500max_iters model ***
****************************************
Average cosine between gold and predicted feature norms: 0.2873325056371902
average Percentage (%) of gold gold-standard features retrieved in 

ModuleNotFoundError: No module named 'lib'

# Results analysis

In [28]:
mcrae_res_df = pd.read_csv('../results/homonymous_words_eval_mcrae_models_expanded_10_20_22.csv')
mcrae_res_df = mcrae_res_df.rename(columns={'Unnamed: 0': 'token_index'})
mcrae_res_df

FileNotFoundError: [Errno 2] No such file or directory: '../results/homonymous_words_eval_mcrae_models_expanded_10_20_22.csv'

In [ ]:
# which model does best?

In [ ]:
# which words get highest precision? lowest?

print(mcrae_res_df.model.unique())

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)


In [ ]:
mcrae_res_df.columns

In [ ]:
# get best and worst performing words on plsr 5k --- avg prec .5
print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.plsr.mc_rae_real.allbuthomoyms.5k.100components.500max_iters'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.plsr.mc_rae_real.allbuthomoyms.5k.100components.500max_iters'
].sort_values(by=['top_k_prec'], ascending=True).head(5))

In [ ]:
# get best and worst performing words on ffnn 1k --- avg prec .5
print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.1k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.1k.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=True).head(5))

In [ ]:
# get best and worst performing words on glove ffnn
print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.ffnn.mc_rae_real.allbuthomoyms.glove.50epochs.0.5dropout.lr1e-4.hsize300'
].sort_values(by=['top_k_prec'], ascending=True).head(10))

In [ ]:
# get best and worst performing words on glove plsr
print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.plsr.mc_rae_real.allbuthomoyms.glove.100components.300max_iters'
].sort_values(by=['top_k_prec'], ascending=False).head(5))

print(mcrae_res_df[mcrae_res_df.model == '../trained_models/model.plsr.mc_rae_real.allbuthomoyms.glove.100components.300max_iters'
].sort_values(by=['top_k_prec'], ascending=True).head(10))

## Look at differential features for tank

In [ ]:
# Lets look side by side at the predictions for a few homonymous pairs, across all models.
# we'll want to look at the gold features, the features predicted, and the scores, for each of the models

# tank
# army = 54
# container = 51

# get gold feats for tank
# print(mcrae_res_df[mcrae_res_df.token_index in (54, 51)][:1][['lemma','context', 'gold_feats']])
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([54,51])][['lemma','context', 'gold_feats']][:2]



In [ ]:
# get predicted feats for each of the models
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([54,51])][['model','lemma','top_10_prec', 'top_10']].sort_values(by='model')


## Look at differentia features for pipe

In [ ]:
# Lets look side by side at the predictions for a few homonymous pairs, across all models.
# we'll want to look at the gold features, the features predicted, and the scores, for each of the models

# pipe
# smoke = 38
# plumbing = 41



In [ ]:
# get predicted feats for each of the models
mcrae_res_df.loc[mcrae_res_df['token_index'].isin([38,41])][['model','lemma','top_10_prec', 'top_10']].sort_values(by='model')


# Correlate predicted features with wu-palmer similarity

use homonym token dataset
 lemma cue_word context 
 
add label row to df

Get predictions on 

In [ ]:
df = pd.read_csv('../data/bnc_contexts_for_mcrae_homonyms.csv')
df.head(5)

In [ ]:
# transform wordnet string into wordnet lemma

ws = []
wn_lemmas = []
for index, row in df.iterrows():
    lemma = re.findall(r"'(.*?)'", row.lemma)[0]
    lemma = wn.lemma(lemma)
    
    word = lemma.name()
    #print(word)
    ws.append(word)
    wn_lemmas.append(lemma)
    
df['label'] = ws
df['wn_lemma'] = wn_lemmas

df.head(5)


In [ ]:
def run_wu_palmer_analysis(df):
    """
    input is a dataframe with columns
        cue_word
        lemma
        context
        label
    """
    wup_sims = []
    cossine_sims = []

    for index, row in df.iterrows():
        word = row.label

        # select other examples of this token
        othertokens = df[df.label == word]
        # filter out the token itself
        othertokens = othertokens[othertokens.index != index]
        #print(othertokens)

        for index, otherword in othertokens.iterrows():
            # find the wordnet distance between these two wordnet senses
            synset1 = row.wn_lemma.synset()
            synset2 = otherword.wn_lemma.synset()

            wup_sim = synset1.wup_similarity(synset2)
            wup_sims.append(wup_sim)
            cossim = 1 - cosine(row.predictions, otherword.predictions)
            #cossim = 1 - cosine(row.single_prototype_model_preds, otherword.single_prototype_model_preds)
            cossine_sims.append(cossim)
            #print(synset1)
            #print(synset2)
            #print(wup_sim)
            #print(cossim)
    return (wup_sims, cossine_sims)

def plot_sims(wup_sims, cossine_sims):
    plt.scatter(wup_sims, cossine_sims)
    plt.title("Wordnet similarity of homonymous senses plotted against cosine similarity of predicted vectors of two tokens in semantic feature space")
    plt.xlabel("Wu and Palmer Similarity")
    plt.ylabel("Cosine Similarity")
    plt.show()

In [ ]:
"""
create a list of predicted feature vectors for each of the models being evaluated
and then stick them in the dataframe
"""

for save_path in mcrae_models:
    print("****************************************")
    print("*** doing wu palmer correlation for %s ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    
    predictions = []
    for index, row in df.iterrows():
            singular = row.wn_lemma.name()
            plural = pluralize(singular)

            try:
                predicted_vector = model.predict_in_context(singular, row.context, bert)
            except:
                predicted_vector = model.predict_in_context(plural, row.context, bert)


            predictions.append(predicted_vector)

    df['predictions'] = predictions
    
    wup_sims, cossine_sims = run_wu_palmer_analysis(df)
    plot_sims(wup_sims, cossine_sims)

    corr, p = pearsonr(wup_sims, cossine_sims)
    print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

    corr, p = spearmanr(wup_sims, cossine_sims)
    print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

# Run the analysis for the non-contextual model (trained on GloVe)

Here we expect to see no correlation, because the model should be making the same prediction for every token of a word form, irrespective of the surrounding context

In [ ]:
"""
create a list of predicted feature vectors for each of the models being evaluated
and then stick them in the dataframe
"""

for save_path in mcrae_glove_models:
    print("****************************************")
    print("*** doing wu palmer correlation for %s ***" % save_path)
    print("****************************************")
    model = torch.load(save_path)
    
    predictions = []
    for index, row in df.iterrows():
            singular = row.wn_lemma.name()
            plural = pluralize(singular)

            try:
                predicted_vector = model.predict_in_context(singular, row.context, bert, glove=True)
            except:
                predicted_vector = model.predict_in_context(plural, row.context, bert, glove=True)


            predictions.append(predicted_vector)

    df['predictions'] = predictions
    
    wup_sims, cossine_sims = run_wu_palmer_analysis(df)
    plot_sims(wup_sims, cossine_sims)

    corr, p = pearsonr(wup_sims, cossine_sims)
    print('Pearsons correlation: %.3f, p-value: %s'  % (corr, p))

    corr, p = spearmanr(wup_sims, cossine_sims)
    print('Spearmans correlation: %.3f, p-value: %s'  % (corr, p))

In [ ]:
embeddings_list = []
word_indexer = Indexer()
with open("../data/glove.6B/glove.6B.300d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_list.append([vector])

        #print(embeddings_dict)
        #raise Exception("hfelfnl")
        word_indexer.add_and_get_index(word)

embs = MultiProtoTypeEmbeddings(word_indexer, np.array(embeddings_list), 0, 1)
